In [4]:
from src import *
from container import Container

c = Container()
Container.wire(c)

rsid = 'rs63750767'
dataframe_generator = NCBIDataFrameGenerator()
rsid_json = dataframe_generator._get_rsid_json_file(rsid)
rsid_data = dataframe_generator._get_data_for_single_rsid(rsid)
allele_annotations = rsid_json['primary_snapshot_data']['allele_annotations']
placements_with_allele = rsid_json['primary_snapshot_data']['placements_with_allele']
result = dataframe_generator._process_allele_annotations(allele_annotations, placements_with_allele)
result

,variant_type,description,deleted,inserted,allele_count,total_count,observed_frequency,diseases,significances,submissions,gene_locus,gene_name
0,dup,dupTCAAAAGGGACATAGAAAA,,TCAAAAGGGACATAGAAAA,3,520284,0.000006,"Hereditary cancer-predisposing syndrome, Lynch...",pathogenic,6,"MSH6, FBXO11","mutS homolog 6, F-box protein 11"
